In [ ]:
"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
Importation of necessary modules
"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pathlib
from numpy import *

from numpy.random import randn
from numpy.random import randint
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import *

from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input

In [ ]:
"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
Downloading the flower dataset
"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
#This allows to download the pictures from tensorflow
#We need to know where it is downloaded so we can use them
URL = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
PIL_file = tf.keras.utils.get_file(origin=URL, fname='flower_photos', untar=True)
PIL_file = pathlib.Path(PIL_file)

#Size chosen for easier convolution : 160 = 5*2^4
height = 160
width = 160

#We are taking all the images of the dataset in one batch because we want to convert them to 
#numpy array in order to use the same code as the Cifar10 dataset 
PIL_dataset = tf.keras.preprocessing.image_dataset_from_directory(PIL_file,image_size=(height, width),batch_size=3670) 


#This will transform the Images into a numpy array that we can feed to our code as is
normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1./127.5, offset=-1)
dataset = PIL_dataset.map(lambda x, y: (normalization_layer(x), y))
dataset,_ = next(iter(dataset))
dataset=dataset.numpy()


Found 3670 files belonging to 5 classes.


In [ ]:
"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
Definition of general variables needed in the code
"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""

ldim = 50 #This is the dimension that will serve to generate the noise that the generator will use : the latent space
epoch=100 #This is the number of times the training process will happen

batch=32 #This is the number of images that we will use for one epoch
#The discriminator will have to judge a batch coming from the dataset and a batch from the generator.
#The generator will create two batches of images

Image_size=(160,160,3) #Setting the image size of the flowers database : (dimension1, dimension2, black&white(1) or color(3))

Optimizer = Adam(lr=0.0002, beta_1=0.5) #Those values are the best ones for an optimizer according to the DCGAN paper
Loss = 'mse' #This is the mean-squared error. To compare with the binary crossentropy results we write 'binary-crossentropy' instead
Activation = 'linear' #This is the activation function for the LSGAN. If we are using binary crossentropy we change it to 'sigmoid'

In [ ]:
"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
Definition of the discriminator model
"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""

discriminator = Sequential()
discriminator.add(Conv2D(64, (5,5), strides=(2, 2), padding='same', input_shape=Image_size))
discriminator.add(LeakyReLU(alpha=0.2))
discriminator.add(Dropout(0.4))

discriminator.add(Conv2D(256, (5,5), strides=(2, 2), padding='same'))
discriminator.add(LeakyReLU(alpha=0.2))
discriminator.add(Dropout(0.4))

discriminator.add(Conv2D(256, (5,5), strides=(2, 2), padding='same'))
discriminator.add(LeakyReLU(alpha=0.2))
discriminator.add(Dropout(0.4))

discriminator.add(Conv2D(256, (5,5), strides=(2, 2), padding='same'))
discriminator.add(LeakyReLU(alpha=0.2))
discriminator.add(Dropout(0.4))

discriminator.add(Conv2D(256, (5,5), strides=(2, 2), padding='same'))
discriminator.add(LeakyReLU(alpha=0.2))
discriminator.add(Dropout(0.4))

discriminator.add(Flatten())
discriminator.add(Dense(1, activation=Activation))

discriminator.compile(loss=Loss, optimizer=Optimizer, metrics=['accuracy'])

In [ ]:
"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
Definition of the generator model
"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""

generator = Sequential()

generator.add(Dense(256*5*5, input_dim=ldim))
generator.add(LeakyReLU(alpha=0.2))
generator.add(Reshape((5, 5, 256)))
 
generator.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
generator.add(LeakyReLU(alpha=0.2))
 
generator.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
generator.add(LeakyReLU(alpha=0.2))

generator.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
generator.add(LeakyReLU(alpha=0.2))
 
generator.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
generator.add(LeakyReLU(alpha=0.2))

generator.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
generator.add(LeakyReLU(alpha=0.2))
 
generator.add(Conv2D(3, (3,3), activation='tanh', padding='same'))

In [ ]:
"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
Definition of the gan model
"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
#This will be used for generator training
#In order to train the generator, we have to put the discriminator as untrainable
#The discriminator is already compiled so it will not change its training
discriminator.trainable = False 

#The gan is simply composed of both the generator and the discriminator
gan = Sequential()
gan.add(generator)
gan.add(discriminator)
gan.compile(loss=Loss, optimizer=Optimizer)

In [ ]:
"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
Training of the models
"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
#We are making a loop to train the model for as many epoch as stated
for i in range(epoch):


#In this GAN, the discriminator and the generator are trained at the same pace 
#which is not necessarily the case for all GANs (example the WGAN)


#####Discriminator training#####

	#Choosing real images randomly from the database
	image_index = randint(0, dataset.shape[0], batch)
	real_images = dataset[image_index] #These are the real images
	real_label = ones((batch, 1)) #1 is the label for real images

	#Generating a random noise as input of the generator in order to generate different images
	noise = randn(ldim * batch)
	noise = noise.reshape(batch, ldim)
	fake_images = generator.predict(noise) #This is the fake images
	fake_label = zeros((batch, 1)) #0 is the label for fake images

	#Stacking the real and fake images together for training of the discriminator
	#This function creates 1 array from 2 arrays
	images = vstack((real_images, fake_images))
	labels = vstack((real_label, fake_label))
 
  #The train_on_batch() function runs a gradient update on this batch of data
  #The discriminator gets images as input data and has to guess the labels correctly
  #This function returns the training loss
	discriminator_loss, _ = discriminator.train_on_batch(images, labels)


#####Generator training#####

	#Noise generation as input of the generator training
	noise = randn(ldim * batch * 2)
	noise = noise.reshape(batch*2, ldim)
	#The generated images should aim to be recognized as real by the discriminator, which will be untrainable during the next step
	real_labelg = ones((batch*2, 1))
 
 #We have the noise as input data and the generator will have to create an image that will be classified as real
	generator_loss = gan.train_on_batch(noise, real_labelg)
 
 #The information of the losses of the discriminator and generator is crucial because it gives us information on the training process
 #If the values are too high or equal to 0, something went wrong and the training should be restarted
	print('%d, d=%.3f, g=%.3f' % (i+1, discriminator_loss, generator_loss))


#####Samples verification#####

	#We should regularly do some checkings on the training to see if it is going in the correct direction
	#We are saving a file with some generated images for inspection
	if (i+1) % 100 == 0 :

		_, accuracy_real = discriminator.evaluate(real_images, real_label, verbose=0) #We evaluate the discriminator on the real images
		_, accuracy_fake = discriminator.evaluate(fake_images, fake_label, verbose=0) #We evaluate the discriminator on the fake images

		#Those are the results of the accuracy of the discriminator. 
		#The accuracy on real values should not be equal to 100%, else the learning process becomes saturated.
		print('Accuracy of the discriminator real: %0.f%%, fake: %0.f%%' % (accuracy_real*100, accuracy_fake*100))
	

		#Saving some generated images to look at them
		examples = (fake_images + 1)/2
		n=3 # n*n will be the number of fake images shown
		for k in range(n*n):
			plt.subplot(n, n, 1 + k)
			plt.axis('off')
			plt.imshow(examples[k])
		plt.savefig('Fake_images%03d' % (i+1))
		plt.close()

1, d=0.530, g=0.987
2, d=0.073, g=0.945
3, d=0.554, g=0.985
4, d=0.138, g=0.986
5, d=0.249, g=0.976
6, d=0.213, g=0.939
7, d=0.069, g=0.798
8, d=0.092, g=0.751
9, d=0.109, g=0.644
10, d=0.135, g=0.852
11, d=0.110, g=1.252
12, d=0.105, g=0.829
13, d=0.085, g=0.934
14, d=0.074, g=1.249
15, d=0.054, g=0.439
16, d=0.377, g=1.648
17, d=0.297, g=0.752
18, d=0.136, g=0.720
19, d=0.223, g=0.751
20, d=0.170, g=0.871
21, d=0.112, g=0.791
22, d=0.119, g=1.165
23, d=0.109, g=0.945
24, d=0.079, g=0.795
25, d=0.103, g=1.467
26, d=0.111, g=1.035
27, d=0.066, g=0.848
28, d=0.130, g=0.829
29, d=0.107, g=0.875
30, d=0.144, g=0.917
31, d=0.079, g=1.007
32, d=0.097, g=0.995
33, d=0.095, g=0.864
34, d=0.102, g=1.257
35, d=0.070, g=1.153
36, d=0.044, g=1.003
37, d=0.039, g=0.919
38, d=0.026, g=0.908
39, d=0.055, g=1.180
40, d=0.030, g=1.150
41, d=0.045, g=0.919
42, d=0.106, g=2.963
43, d=0.135, g=1.047
44, d=0.104, g=0.916
45, d=0.124, g=0.740
46, d=0.088, g=0.505
47, d=0.191, g=0.707
48, d=0.159, g=0.638
4

In [ ]:
"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
Calculating the Inception Score
"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""

#We are shaping the last results in order to evaluate their inception score
result=(examples*127.5+127.5).astype('float32')

#InceptionV3() is an image recognition neural network which is used to calculate the inception score
model = InceptionV3()

#The InceptionV3() function accepts only entry in the shape of (299,299,3), so we need to resize all of our images to this shape
data_list = list()
for i in result:
	data = resize(i, (299,299,3))
	data_list.append(data)
 
result=asarray(data_list)

#We need to preprocess the data so that the predict() function from InceptionV3 can work
#It gives us a probability distribution that needs to be transformed in order to get the inception score
#This probability distribution depends on the number of classes perceived by the algorithm

P_yx = model.predict(preprocess_input(result))

#This is the probability that each image is in a class. The sum of this probability for an image is 1
P_y = P_yx.mean(axis=0)

#We are using the Kullback-Leibler divergence with logarithmic probabilities
#This will give us the randomness of the probability we obtained; the less random the higher the inception score will be
#We need the 'e' value in case the probability is 0, which would cause an infinity error
e=1E-16
KL = P_yx * (log(P_yx + e) - log(P_y + e))

#We are making the sum of the values for each image
sum_KL = KL.sum(axis=1)
# Averaging on all the images and adding an exponential to counteract the previous logarithm
#If the average of the sum is close to zero, it means no structure has been detected and the IS will be close to one
IS = exp(mean(sum_KL))

print('Inception score : ', IS)

Inception score :  1.0617894


In [ ]:
# define the standalone discriminator model
def define_discriminator(in_shape=(160,160,3)):
	model = Sequential()
	model.add(Conv2D(64, (5,5), strides=(2, 2), padding='same', input_shape=in_shape))
	model.add(LeakyReLU(alpha=0.2))
	model.add(Dropout(0.4))
 
	model.add(Conv2D(256, (5,5), strides=(2, 2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	model.add(Dropout(0.4))
	model.add(BatchNormalization())
	
	model.add(Conv2D(256, (5,5), strides=(2, 2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	model.add(Dropout(0.4))
	model.add(BatchNormalization())

	model.add(Conv2D(256, (5,5), strides=(2, 2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	model.add(Dropout(0.4))

	model.add(Conv2D(256, (5,5), strides=(2, 2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	model.add(Dropout(0.4))

	model.add(Flatten())
	model.add(Dense(1, activation='linear'))    #change to 'sigmoid' for binary-crossentropy

	opt = Adam(lr=0.0002, beta_1=0.5)
	model.compile(loss='mse', optimizer=opt, metrics=['accuracy'])   #change to 'binary-crossentropy'
	return model
 
# define the standalone generator model
def define_generator(latent_dim):
	model = Sequential()
 
	# foundation for 5x5 image
	n_nodes = 256 * 5 * 5
	model.add(Dense(n_nodes, input_dim=latent_dim))
	model.add(LeakyReLU(alpha=0.2))
	model.add(Reshape((5, 5, 256)))
 
	# upsample to 10x10
	model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
 
		# upsample to 20x20
	model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
 
		# upsample to 40x40
	model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
 
		# upsample to 80x80
	model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
 
		# upsample to 160x160
	model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
 

	model.add(Conv2D(3, (3,3), activation='tanh', padding='same'))
	return model
 
# define the combined generator and discriminator model, for updating the generator
def define_gan(g_model, d_model):
	d_model.trainable = False
	model = Sequential()
	model.add(g_model)
	model.add(d_model)
	opt = Adam(lr=0.0002, beta_1=0.5)
	model.compile(loss='mse', optimizer=opt)    #change to 'binary-crossentropy'
	return model
 
# load and prepare cifar10 training images
def load_real_samples():
	X = trainX.astype('float32')
	return X
 
# select real samples
def generate_real_samples(dataset, n_samples):
	# choose random instances
	ix = randint(0, dataset.shape[0], n_samples)
	# retrieve selected images
	X = dataset[ix]
	# generate 'real' class labels (1)
	y = ones((n_samples, 1))
	return X, y
 
# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples):
	# generate points in the latent space
	x_input = randn(latent_dim * n_samples)
	# reshape into a batch of inputs for the network
	x_input = x_input.reshape(n_samples, latent_dim)
	return x_input
 
# use the generator to generate n fake examples, with class labels
def generate_fake_samples(g_model, latent_dim, n_samples):
	# generate points in latent space
	x_input = generate_latent_points(latent_dim, n_samples)
	# predict outputs
	X = g_model.predict(x_input)
	# create 'fake' class labels (0)
	y = zeros((n_samples, 1))
	return X, y
 
# create and save a plot of generated images
def save_plot(examples, epoch, n=2):
	# scale from [-1,1] to [0,1]
	examples = (examples + 1) / 2.0
	# plot images
	for i in range(n * n):
		# define subplot
		plt.subplot(n, n, 1 + i)
		# turn off axis
		plt.axis('off')
		# plot raw pixel data
		plt.imshow(examples[i])
	# save plot to file
	filename = 'generated_plot_e%03d.png' % (epoch+1)
	plt.savefig(filename)
	plt.close()
 
# evaluate the discriminator, plot generated images, save generator model
def summarize_performance(epoch, g_model, d_model, dataset, latent_dim, n_samples=2000):
	# prepare real samples
	X_real, y_real = generate_real_samples(dataset, n_samples)
	# evaluate discriminator on real examples
	_, acc_real = d_model.evaluate(X_real, y_real, verbose=0)
	# prepare fake examples
	x_fake, y_fake = generate_fake_samples(g_model, latent_dim, n_samples)
	# evaluate discriminator on fake examples
	_, acc_fake = d_model.evaluate(x_fake, y_fake, verbose=0)
	# summarize discriminator performance
	print('>Accuracy real: %.0f%%, fake: %.0f%%' % (acc_real*100, acc_fake*100))
	# save plot
	save_plot(x_fake, epoch)
	# save the generator model tile file
	filename = 'generator_model_%03d.h5' % (epoch+1)
	g_model.save(filename)
	return x_fake

# train the generator and discriminator
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=800, n_batch=32):
	bat_per_epo = int(dataset.shape[0] / n_batch)
	half_batch = int(n_batch / 2)
	fake_image=trainX[0]
	fake_images=[]
	# manually enumerate epochs
	for i in range(n_epochs):
		# enumerate batches over the training set
		for j in range(bat_per_epo):
			# get randomly selected 'real' samples
			X_real, y_real = generate_real_samples(dataset, half_batch)
			# generate 'fake' examples
			X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
			# create training set for the discriminator
			X, y = vstack((X_real, X_fake)), vstack((y_real, y_fake))
			# update discriminator model weights
			d_loss, _ = d_model.train_on_batch(X, y)
			# prepare points in latent space as input for the generator
			X_gan = generate_latent_points(latent_dim, n_batch)
			# create inverted labels for the fake samples
			y_gan = ones((n_batch, 1))
			# update the generator via the discriminator's error
			g_loss = gan_model.train_on_batch(X_gan, y_gan)
			# summarize loss on this batch
			print('>%d, %d/%d, d=%.3f, g=%.3f' % (i+1, j+1, bat_per_epo, d_loss, g_loss))
		# evaluate the model performance, sometimes
		if (i+1) % 20 == 0:
			fake_image = (summarize_performance(i, g_model, d_model, dataset, latent_dim))
		if (i+1) % 1 == 0 and i>=797:
			fake_image = (summarize_performance(i, g_model, d_model, dataset, latent_dim))
			fake_images.append(fake_image)
	return fake_images    #This is used to retrieve the Inception Score
 
# size of the latent space
latent_dim = 50
# create the discriminator
d_model = define_discriminator()
# create the generator
g_model = define_generator(latent_dim)
# create the gan
gan_model = define_gan(g_model, d_model)
# load image data
dataset = trainX.astype('float32')
print(dataset.shape)
# train model
fake_images=train(g_model, d_model, gan_model, dataset, latent_dim)

(630, 160, 160, 3)


KeyboardInterrupt: ignored

In [ ]:
import numpy
from math import floor
from numpy import ones
from numpy import expand_dims
from numpy import log
from numpy import mean
from numpy import std
from numpy import exp
from numpy.random import shuffle
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input
from keras.datasets import cifar10
from skimage.transform import resize
from numpy import asarray
 
# scale an array of images to a new size
def scale_images(images, new_shape):
	images_list = list()
	for image in images:
		# resize with nearest neighbor interpolation
		new_image = resize(image, new_shape, 0)
		# store
		images_list.append(new_image)
	return asarray(images_list)
 
# assumes images have any shape and pixels in [0,255]
def calculate_inception_score(images, n_split=10, eps=1E-16):
	# load inception v3 model
	model = InceptionV3()
	# enumerate splits of images/predictions
	scores = list()
	n_part = floor(images.shape[0] / n_split)
	for i in range(n_split):
		# retrieve images
		ix_start, ix_end = i * n_part, (i+1) * n_part
		subset = images[ix_start:ix_end]
		# convert from uint8 to float32
		subset = subset.astype('float32')
		# scale images to the required size
		subset = scale_images(subset, (299,299,3))
		# pre-process images, scale to [-1,1]
		subset = preprocess_input(subset)
		# predict p(y|x)
		p_yx = model.predict(subset)
		# calculate p(y)
		p_y = expand_dims(p_yx.mean(axis=0), 0)
		# calculate KL divergence using log probabilities
		kl_d = p_yx * (log(p_yx + eps) - log(p_y + eps))
		# sum over classes
		sum_kl_d = kl_d.sum(axis=1)
		# average over images
		avg_kl_d = mean(sum_kl_d)
		# undo the log
		is_score = exp(avg_kl_d)
		# store
		scores.append(is_score)
	# average across images
	is_avg, is_std = mean(scores), std(scores)
	return is_avg, is_std
for i in range(0, 9):
	plt.subplot(3,3, 1+i)
	plt.axis('off')
	plt.imshow(fake_images[i])
fake_images=result
print(fake_images*128+128) #Convert to [0,255] for image representation

# calculate inception score
is_avg, is_std = calculate_inception_score(fake_images*128+128)
print('score', is_avg, is_std)

NameError: ignored